In [2]:
import timeit
from qsc import Qsc
from qsc.calculate_r3 import calc_r3_new 
from qsc.calculate_r1 import r1_diagnostics

stel = Qsc(rc=[1, 0.09], zs=[0, -0.09], nfp=2, etabar=0.95, I2=0.9, order='r2', B2c=-0.7, p2=-600000.)




ImportError: cannot import name 'Qsc' from 'qsc' (/Users/z/Documents/GitHub/pyNACX/qsc/__init__.py)

In [ ]:

timeit.timeit(lambda: calc_r3_new(stel.B0, stel.G0, stel.X20, stel.Y1c, stel.X2c, stel.X2s, stel.etabar*stel.B0, stel.X1c, stel.X1s, stel.Y1s, stel.I2, stel.iotaN, stel.B20, stel.Y20, stel.Y2c, stel.Y2s, stel.Z20, stel.abs_G0_over_B0, stel.Z2c, stel.Z2s, stel.torsion, stel.d_X1c_d_varphi, stel.d_Y1c_d_varphi, stel.d_d_varphi, stel.spsi, stel.p2, stel.curvature, stel.d_Z20_d_varphi, stel.sG, stel.G2, stel.N_helicity, stel.helicity, stel.nfp, stel.varphi), number=10000)

In [20]:
import numpy as np

iterations = 10**(np.arange(6))
results = np.zeros((3, len(iterations)))

for i, num in enumerate(iterations):
    results[0][i] = timeit.timeit(lambda: r1_diagnostics(stel.nfp, stel.etabar, stel.sG, stel.spsi, stel.curvature, stel.sigma, stel.helicity, stel.varphi, stel.X1s, stel.X1c, stel.d_l_d_phi, stel.d_d_varphi, stel.B0, stel.d_l_d_varphi, stel.tangent_cylindrical, stel.normal_cylindrical, stel.binormal_cylindrical, stel.iotaN, stel.torsion), number=num)
    results[1][i] = timeit.timeit(lambda: calc_r3_new(stel.B0, stel.G0, stel.X20, stel.Y1c, stel.X2c, stel.X2s, stel.etabar*stel.B0, stel.X1c, stel.X1s, stel.Y1s, stel.I2, stel.iotaN, stel.B20, stel.Y20, stel.Y2c, stel.Y2s, stel.Z20, stel.abs_G0_over_B0, stel.Z2c, stel.Z2s, stel.torsion, stel.d_X1c_d_varphi, stel.d_Y1c_d_varphi, stel.d_d_varphi, stel.spsi, stel.p2, stel.curvature, stel.d_Z20_d_varphi, stel.sG, stel.G2, stel.N_helicity, stel.helicity, stel.nfp, stel.varphi), number=num)
    


factor 0.9841224002470026
tn [1.18803769 1.18777624 1.18697879 1.18560608 1.18359298 1.18084901
 1.17725911 1.17268491 1.16696642 1.15992447 1.15136406 1.14107866
 1.12885598 1.11448513 1.09776569 1.07851875 1.05660019 1.03191645
 1.00444293 0.97424506 0.94150214 0.90653365 0.86982744 0.8320683
 0.79416369 0.75726037 0.72274063 0.69218101 0.66725203 0.64954433
 0.64033242 0.64033242 0.64954433 0.66725203 0.69218101 0.72274063
 0.75726037 0.79416369 0.8320683  0.86982744 0.90653365 0.94150214
 0.97424506 1.00444293 1.03191645 1.05660019 1.07851875 1.09776569
 1.11448513 1.12885598 1.14107866 1.15136406 1.15992447 1.16696642
 1.17268491 1.17725911 1.18084901 1.18359298 1.18560608 1.18697879
 1.18777624]
nt [1.18803769 1.18777624 1.18697879 1.18560608 1.18359298 1.18084901
 1.17725911 1.17268491 1.16696642 1.15992447 1.15136406 1.14107866
 1.12885598 1.11448513 1.09776569 1.07851875 1.05660019 1.03191645
 1.00444293 0.97424506 0.94150214 0.90653365 0.86982744 0.8320683
 0.79416369 0.75726

KeyboardInterrupt: 